# CROWD ~

## Тут было бы неплохо вставить описание

In [1]:
#Вспомогательное
import pandas as pd # для работы с датасетами
import numpy as np # для работы с листами, математическая обработка
from sklearn.metrics import accuracy_score # точность совпадения
from sklearn import preprocessing # предобработка датасета
from sklearn import metrics # различные метрики
from sklearn.model_selection import train_test_split # для разбиения на обучающую и тестовую выборки
from sklearn.model_selection import cross_val_score # ну всё понятно, умное слово
#Дерево
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
#Линейная регрессия
from sklearn.linear_model import LinearRegression # модель линейной
#Логическая регрессия
from sklearn.linear_model import LogisticRegression # модель логический
#Нейронка
from keras.models import Sequential # очень важная штука
from keras.layers import Dense # создает слой нейронов

Using TensorFlow backend.


### Обработка датасета для работы с ним

In [2]:
df = pd.read_csv("temp.csv", sep=";").drop(["Unnamed: 0"], axis=1)

In [3]:
df

,date,year,mounth,day,weekday,temp,pres,prec,wind,absenteeism,...,1,2,3,4,5,6,7,8,9,10
0,20120114,2012,1,14,6,-21,756,0,1.0,4,...,0,0,0,1,0,0,0,0,0,0
1,20120116,2012,1,16,1,-19,758,0,2.0,3,...,0,0,1,0,0,0,0,0,0,0
2,20120117,2012,1,17,2,-27,770,0,1.0,3,...,0,0,1,0,0,0,0,0,0,0
3,20120118,2012,1,18,3,-28,776,0,2.0,6,...,0,0,0,0,0,1,0,0,0,0
4,20120119,2012,1,19,4,-27,778,0,2.0,4,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1689,20181225,2018,12,25,2,-23,770,0,1.0,2,...,0,1,0,0,0,0,0,0,0,0
1690,20181226,2018,12,26,3,-22,770,0,NaN,1,...,1,0,0,0,0,0,0,0,0,0
1691,20181227,2018,12,27,4,-23,770,0,NaN,1,...,1,0,0,0,0,0,0,0,0,0
1692,20181228,2018,12,28,5,-25,766,0,NaN,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
X = df[["year", "mounth", "day", "weekday", "temp", "pres", "prec", "wind"]]
X = X.apply(pd.to_numeric).fillna(0).astype("int")

In [5]:
X

,year,mounth,day,weekday,temp,pres,prec,wind
0,2012,1,14,6,-21,756,0,1
1,2012,1,16,1,-19,758,0,2
2,2012,1,17,2,-27,770,0,1
3,2012,1,18,3,-28,776,0,2
4,2012,1,19,4,-27,778,0,2
...,...,...,...,...,...,...,...,...
1689,2018,12,25,2,-23,770,0,1
1690,2018,12,26,3,-22,770,0,0
1691,2018,12,27,4,-23,770,0,0
1692,2018,12,28,5,-25,766,0,0


In [6]:
y = df[["absenteeism","0", "1", "2", "3", "4", "5", "6", "7", "8", "9", "10"]].astype("int")

In [7]:
y

,absenteeism,0,1,2,3,4,5,6,7,8,9,10
0,4,0,0,0,0,1,0,0,0,0,0,0
1,3,0,0,0,1,0,0,0,0,0,0,0
2,3,0,0,0,1,0,0,0,0,0,0,0
3,6,0,0,0,0,0,0,1,0,0,0,0
4,4,0,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
1689,2,0,0,1,0,0,0,0,0,0,0,0
1690,1,0,1,0,0,0,0,0,0,0,0,0
1691,1,0,1,0,0,0,0,0,0,0,0,0
1692,0,1,0,0,0,0,0,0,0,0,0,0


In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y)

Тут мы делим **y** на абсолютное и матричное значение:

In [9]:
y_train_value, y_test_value = y_train["absenteeism"], y_test["absenteeism"]

In [10]:
y_test_matrix, y_train_matrix = y_test.drop(["absenteeism"], axis=1), y_train.drop(["absenteeism"], axis=1)

### Вспомогательные функции

In [11]:
def show_delta(real,predicted,toint=True): # показывает, на сколько отличается реальное от предсказанного.
    if toint:
        delta = [[i[0], int(i[1]), int(i[1]) - i[0]] for i in list(zip(real, predicted))]
    else:
        delta = [[i[0], round(i[1],2), round(i[1] - i[0],2)] for i in list(zip(real, predicted))]
    return delta

# На основе значения

### Дерево

In [12]:
kfold = StratifiedKFold(n_splits=3) # задаем объект для разделения данных на фолды при кросс-валидации

model = DecisionTreeClassifier()

params = {
          'max_depth': range(1,50,2)
         }  # перебираемые параметры модели

grid = GridSearchCV(model, params, cv=kfold, scoring='accuracy', n_jobs=-1)

In [13]:
grid.fit(X_train, y_train_value)

GridSearchCV(cv=StratifiedKFold(n_splits=3, random_state=None, shuffle=False),
             error_score=nan,
             estimator=DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort='deprecated',
                                              random_state=None,
                                              splitter='best'),
             iid='deprecated', n_jobs=-1,
       

In [14]:
mod = grid.best_estimator_
mod

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=5, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [15]:
y_pred_tree = mod.predict(X_test)

In [16]:
print('\nScore test: \n', mod.score(X_test, y_test_value))
print('\nScore train: \n', mod.score(X_train, y_train_value))
print('\nMean absolute error: \n', metrics.mean_absolute_error(y_test_value, y_pred_tree))
print('\nMedian absolute error: \n', metrics.median_absolute_error(y_test_value, y_pred_tree), "\n")
print(*show_delta(y_test_value, y_pred_tree), sep="\n")


Score test: 
 0.381139489194499

Score train: 
 0.45147679324894513

Mean absolute error: 
 0.9155206286836935

Median absolute error: 
 1.0 

[4, 4, 0]
[1, 2, 1]
[2, 3, 1]
[2, 3, 1]
[1, 1, 0]
[4, 3, -1]
[1, 1, 0]
[4, 4, 0]
[0, 0, 0]
[3, 3, 0]
[0, 0, 0]
[4, 3, -1]
[3, 3, 0]
[0, 0, 0]
[3, 3, 0]
[1, 1, 0]
[4, 4, 0]
[4, 3, -1]
[4, 4, 0]
[4, 7, 3]
[2, 3, 1]
[2, 1, -1]
[3, 3, 0]
[3, 3, 0]
[3, 2, -1]
[2, 4, 2]
[0, 1, 1]
[1, 1, 0]
[0, 3, 3]
[2, 3, 1]
[5, 4, -1]
[4, 3, -1]
[5, 1, -4]
[1, 1, 0]
[2, 2, 0]
[2, 3, 1]
[1, 2, 1]
[2, 2, 0]
[0, 2, 2]
[4, 3, -1]
[3, 3, 0]
[3, 3, 0]
[2, 1, -1]
[5, 4, -1]
[1, 1, 0]
[4, 3, -1]
[6, 7, 1]
[3, 3, 0]
[2, 2, 0]
[4, 3, -1]
[4, 4, 0]
[2, 3, 1]
[2, 2, 0]
[2, 2, 0]
[2, 3, 1]
[4, 4, 0]
[4, 3, -1]
[4, 3, -1]
[5, 4, -1]
[2, 3, 1]
[4, 3, -1]
[3, 3, 0]
[2, 3, 1]
[5, 3, -2]
[2, 2, 0]
[0, 0, 0]
[3, 3, 0]
[3, 1, -2]
[1, 0, -1]
[3, 3, 0]
[3, 3, 0]
[7, 3, -4]
[4, 3, -1]
[4, 2, -2]
[2, 3, 1]
[2, 4, 2]
[3, 2, -1]
[3, 3, 0]
[4, 3, -1]
[0, 2, 2]
[7, 7, 0]
[3, 3, 0]
[4, 3, -1]


# Линейная регрессия

In [17]:
LR = LinearRegression()
model_lin = LR.fit(X_train, y_train_value)
y_pred_lin = model_lin.predict(X_test)

In [18]:
linear_scoring = cross_val_score(model_lin, X_train, y_train_value, scoring="neg_mean_absolute_error", cv=10)
print(f"{linear_scoring.mean()}, {linear_scoring.std()}")

-1.0597108222699325, 0.07921764172043846


In [19]:
print('\nIntercept: \n', model_lin.intercept_)
print('\nScore test: \n', model_lin.score(X_test, y_test_value))
print('\nScore train: \n', model_lin.score(X_train, y_train_value))
print('\nMean absolute error: \n', metrics.mean_absolute_error(y_test_value, y_pred_lin))
print('\nMedian absolute error: \n', metrics.median_absolute_error(y_test_value, y_pred_lin), "\n")
print(*show_delta(y_test_value, y_pred_lin, toint=False), sep="\n")


Intercept: 
 -557.0953473298142

Score test: 
 0.4114093735304373

Score train: 
 0.36091430631649557

Mean absolute error: 
 1.0342071227623129

Median absolute error: 
 0.8381726760599122 

[4, 2.96, -1.04]
[1, 1.58, 0.58]
[2, 1.68, -0.32]
[2, 1.79, -0.21]
[1, 1.64, 0.64]
[4, 3.22, -0.78]
[1, 1.01, 0.01]
[4, 4.38, 0.38]
[0, 1.09, 1.09]
[3, 3.82, 0.82]
[0, 1.16, 1.16]
[4, 1.87, -2.13]
[3, 2.59, -0.41]
[0, 1.18, 1.18]
[3, 3.54, 0.54]
[1, 1.98, 0.98]
[4, 4.43, 0.43]
[4, 2.33, -1.67]
[4, 4.43, 0.43]
[4, 4.38, 0.38]
[2, 2.96, 0.96]
[2, 1.61, -0.39]
[3, 3.2, 0.2]
[3, 3.75, 0.75]
[3, 2.35, -0.65]
[2, 3.73, 1.73]
[0, 2.03, 2.03]
[1, 2.13, 1.13]
[0, 2.73, 2.73]
[2, 2.68, 0.68]
[5, 5.19, 0.19]
[4, 3.48, -0.52]
[5, 2.62, -2.38]
[1, 1.73, 0.73]
[2, 0.92, -1.08]
[2, 3.47, 1.47]
[1, 1.7, 0.7]
[2, 1.91, -0.09]
[0, 2.02, 2.02]
[4, 2.82, -1.18]
[3, 3.64, 0.64]
[3, 4.13, 1.13]
[2, 1.7, -0.3]
[5, 4.36, -0.64]
[1, 1.73, 0.73]
[4, 4.07, 0.07]
[6, 4.44, -1.56]
[3, 2.29, -0.71]
[2, 1.58, -0.42]
[4, 2.99, 

In [20]:
model_lin.predict([[2018, 12, 6, 5, -21, 770, 0, 1]])

array([4.70852613])

---

## Матричные

### Нейронка

In [21]:
def nn_num(X_train_local, y_train_local, epochs=1000, batch=12):
    model = Sequential()
    model.add(Dense(12, input_dim=len(list(X_train_local)), activation='relu'))
    model.add(Dense(122, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss="binary_crossentropy", optimizer="adam", metrics=['accuracy'])
    model.fit(X_train_local, y_train_local, epochs = epochs, batch_size=batch)
    return model

In [22]:
models_nn = [None]*len(list(y_train_matrix))
for i in range(len(models_nn)):
    models_nn[i] = nn_num(X_train, y_train_matrix[str(i)], epochs=10, batch=40)

Epoch 1/10
1185/1185 [==============================] - 0s 121us/step - loss: 12.3946 - accuracy: 0.8295
Epoch 2/10
1185/1185 [==============================] - 0s 29us/step - loss: 4.3553 - accuracy: 0.8262
Epoch 3/10
1185/1185 [==============================] - 0s 26us/step - loss: 0.4826 - accuracy: 0.8726
Epoch 4/10
1185/1185 [==============================] - 0s 25us/step - loss: 0.9656 - accuracy: 0.8456
Epoch 5/10
1185/1185 [==============================] - 0s 25us/step - loss: 1.4173 - accuracy: 0.8405
Epoch 6/10
1185/1185 [==============================] - 0s 24us/step - loss: 1.2403 - accuracy: 0.8329
Epoch 7/10
1185/1185 [==============================] - 0s 26us/step - loss: 1.7096 - accuracy: 0.8245
Epoch 8/10
1185/1185 [==============================] - 0s 28us/step - loss: 1.4353 - accuracy: 0.8228
Epoch 9/10
1185/1185 [==============================] - 0s 28us/step - loss: 2.0887 - accuracy: 0.8489
Epoch 10/10
1185/1185 [==============================] - 0s 31us/step -

1185/1185 [==============================] - 0s 25us/step - loss: 0.0808 - accuracy: 0.9857
Epoch 1/10
1185/1185 [==============================] - 0s 102us/step - loss: 0.7243 - accuracy: 0.9629
Epoch 2/10
1185/1185 [==============================] - 0s 32us/step - loss: 0.8888 - accuracy: 0.9932
Epoch 3/10
1185/1185 [==============================] - 0s 29us/step - loss: 0.4728 - accuracy: 0.9932
Epoch 4/10
1185/1185 [==============================] - 0s 26us/step - loss: 0.1556 - accuracy: 0.9873
Epoch 5/10
1185/1185 [==============================] - 0s 25us/step - loss: 0.0723 - accuracy: 0.9890
Epoch 6/10
1185/1185 [==============================] - 0s 25us/step - loss: 0.1802 - accuracy: 0.9890
Epoch 7/10
1185/1185 [==============================] - 0s 24us/step - loss: 0.1864 - accuracy: 0.9899
Epoch 8/10
1185/1185 [==============================] - 0s 28us/step - loss: 0.0901 - accuracy: 0.9907
Epoch 9/10
1185/1185 [==============================] - 0s 27us/step - loss: 0.0874

In [23]:
predicted_nn = [None]*len(models_nn)
for model in enumerate(models_nn):
    predicted_nn[model[0]] = model[1].predict(X_test, batch_size=20)
y_pred_nn = [[round(float(j[i][0]),2) for j in list(predicted_nn)] for i in range(len(predicted_nn[0]))]

In [24]:
y_pred_nn

[[0.0, 0.0, 0.05, 0.19, 0.05, 0.03, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.04, 0.23, 0.05, 0.03, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.02, 0.15, 0.03, 0.02, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.02, 0.12, 0.01, 0.01, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.01, 0.13, 0.01, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.05, 0.02, 0.02, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.01, 0.1, 0.01, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.01, 0.11, 0.08, 0.06, 0.01, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.04, 0.26, 0.04, 0.02, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.01, 0.06, 0.09, 0.05, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.01, 0.07, 0.02, 0.01, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.01, 0.12, 0.02, 0.01, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.01, 0.21, 0.01, 0.01, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.01, 0.06, 0.01, 0.01, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.08, 0.02, 0.02, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.01, 0.11, 0.01, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0

In [25]:
y_pred_nn_value = [i.index(max(i)) for i in y_pred_nn]

In [26]:
show_delta(list(y_test["absenteeism"]), y_pred_nn_value)

[[4, 3, -1],
 [1, 3, 2],
 [2, 3, 1],
 [2, 3, 1],
 [1, 3, 2],
 [4, 3, -1],
 [1, 3, 2],
 [4, 3, -1],
 [0, 3, 3],
 [3, 4, 1],
 [0, 3, 3],
 [4, 3, -1],
 [3, 3, 0],
 [0, 3, 3],
 [3, 3, 0],
 [1, 3, 2],
 [4, 3, -1],
 [4, 3, -1],
 [4, 3, -1],
 [4, 3, -1],
 [2, 3, 1],
 [2, 3, 1],
 [3, 3, 0],
 [3, 3, 0],
 [3, 3, 0],
 [2, 3, 1],
 [0, 3, 3],
 [1, 3, 2],
 [0, 3, 3],
 [2, 3, 1],
 [5, 3, -2],
 [4, 3, -1],
 [5, 3, -2],
 [1, 3, 2],
 [2, 3, 1],
 [2, 4, 2],
 [1, 3, 2],
 [2, 3, 1],
 [0, 3, 3],
 [4, 3, -1],
 [3, 3, 0],
 [3, 3, 0],
 [2, 3, 1],
 [5, 3, -2],
 [1, 3, 2],
 [4, 3, -1],
 [6, 3, -3],
 [3, 3, 0],
 [2, 3, 1],
 [4, 3, -1],
 [4, 3, -1],
 [2, 3, 1],
 [2, 3, 1],
 [2, 3, 1],
 [2, 3, 1],
 [4, 3, -1],
 [4, 3, -1],
 [4, 3, -1],
 [5, 3, -2],
 [2, 3, 1],
 [4, 3, -1],
 [3, 3, 0],
 [2, 3, 1],
 [5, 3, -2],
 [2, 3, 1],
 [0, 3, 3],
 [3, 3, 0],
 [3, 3, 0],
 [1, 3, 2],
 [3, 3, 0],
 [3, 3, 0],
 [7, 3, -4],
 [4, 3, -1],
 [4, 3, -1],
 [2, 3, 1],
 [2, 4, 2],
 [3, 3, 0],
 [3, 3, 0],
 [4, 3, -1],
 [0, 3, 3],
 [7, 3, -4],


In [27]:
print('\nScore test: \n', accuracy_score(list(y_test["absenteeism"]), [i.index(max(i)) for i in y_pred_nn]))
print('\nMean absolute error: \n', metrics.mean_absolute_error(y_test_value, y_pred_nn_value))
print('\nMedian absolute error: \n', metrics.median_absolute_error(y_test_value, y_pred_nn_value), "\n")
print(*show_delta(y_test_value, y_pred_nn_value), sep="\n")


Score test: 
 0.27897838899803534

Mean absolute error: 
 1.2789783889980353

Median absolute error: 
 1.0 

[4, 3, -1]
[1, 3, 2]
[2, 3, 1]
[2, 3, 1]
[1, 3, 2]
[4, 3, -1]
[1, 3, 2]
[4, 3, -1]
[0, 3, 3]
[3, 4, 1]
[0, 3, 3]
[4, 3, -1]
[3, 3, 0]
[0, 3, 3]
[3, 3, 0]
[1, 3, 2]
[4, 3, -1]
[4, 3, -1]
[4, 3, -1]
[4, 3, -1]
[2, 3, 1]
[2, 3, 1]
[3, 3, 0]
[3, 3, 0]
[3, 3, 0]
[2, 3, 1]
[0, 3, 3]
[1, 3, 2]
[0, 3, 3]
[2, 3, 1]
[5, 3, -2]
[4, 3, -1]
[5, 3, -2]
[1, 3, 2]
[2, 3, 1]
[2, 4, 2]
[1, 3, 2]
[2, 3, 1]
[0, 3, 3]
[4, 3, -1]
[3, 3, 0]
[3, 3, 0]
[2, 3, 1]
[5, 3, -2]
[1, 3, 2]
[4, 3, -1]
[6, 3, -3]
[3, 3, 0]
[2, 3, 1]
[4, 3, -1]
[4, 3, -1]
[2, 3, 1]
[2, 3, 1]
[2, 3, 1]
[2, 3, 1]
[4, 3, -1]
[4, 3, -1]
[4, 3, -1]
[5, 3, -2]
[2, 3, 1]
[4, 3, -1]
[3, 3, 0]
[2, 3, 1]
[5, 3, -2]
[2, 3, 1]
[0, 3, 3]
[3, 3, 0]
[3, 3, 0]
[1, 3, 2]
[3, 3, 0]
[3, 3, 0]
[7, 3, -4]
[4, 3, -1]
[4, 3, -1]
[2, 3, 1]
[2, 4, 2]
[3, 3, 0]
[3, 3, 0]
[4, 3, -1]
[0, 3, 3]
[7, 3, -4]
[3, 3, 0]
[4, 3, -1]
[3, 3, 0]
[3, 3, 0]
[3, 3, 0]
[

### Логическая регрессия. Почти ничем не отличается от нейронки

In [28]:
def logreg_num(X_train_local, y_train_local):
    LR = LinearRegression()
    model = LR.fit(X_train_local, y_train_local)
    return model

In [29]:
models_logreg = [None]*len(list(y_train_matrix))
for i in range(len(models_logreg)):
    models_logreg[i] = logreg_num(X_train, y_train_matrix[str(i)])

In [30]:
predicted_logreg = [None]*len(models_logreg)
for model in enumerate(models_logreg):
    predicted_logreg[model[0]] = model[1].predict(X_test)
y_pred_logreg = [[round(float(j[i]),2) for j in list(predicted_logreg)] for i in range(len(predicted_logreg[0]))]

In [31]:
y_pred_logreg

[[0.08, 0.13, 0.09, 0.3, 0.25, 0.12, 0.03, 0.01, 0.01, 0.0, -0.01],
 [0.17, 0.2, 0.32, 0.28, 0.09, -0.0, -0.0, -0.02, -0.02, -0.01, -0.01],
 [0.23, 0.2, 0.27, 0.18, 0.09, 0.03, 0.02, -0.02, -0.01, -0.01, -0.01],
 [0.16, 0.21, 0.23, 0.32, 0.12, -0.0, -0.01, -0.02, 0.01, -0.01, -0.01],
 [0.17, 0.22, 0.25, 0.33, 0.11, -0.03, -0.02, -0.03, 0.01, -0.01, -0.0],
 [-0.0, 0.09, 0.25, 0.34, 0.18, 0.04, 0.06, 0.02, 0.01, 0.0, 0.01],
 [0.28, 0.26, 0.31, 0.22, 0.03, -0.04, -0.02, -0.03, 0.0, -0.01, -0.0],
 [-0.04, -0.01, 0.16, 0.26, 0.27, 0.18, 0.09, 0.06, 0.0, 0.03, 0.01],
 [0.27, 0.23, 0.37, 0.18, 0.03, -0.0, -0.02, -0.03, -0.02, -0.0, -0.0],
 [0.06, 0.07, 0.1, 0.24, 0.19, 0.14, 0.07, 0.06, 0.02, 0.02, 0.01],
 [0.31, 0.28, 0.26, 0.17, 0.02, -0.01, -0.01, -0.02, 0.01, -0.01, -0.0],
 [0.26, 0.19, 0.24, 0.17, 0.06, 0.03, 0.02, -0.0, 0.01, 0.01, 0.0],
 [0.12, 0.11, 0.26, 0.25, 0.17, 0.06, 0.03, -0.01, -0.0, 0.0, 0.0],
 [0.29, 0.26, 0.31, 0.17, 0.01, -0.02, -0.01, -0.02, 0.01, -0.01, 0.0],
 [0.03, 0.0

In [32]:
show_delta(list(y_test["absenteeism"]),[i.index(max(i)) for i in y_pred_logreg])

[[4, 3, -1],
 [1, 2, 1],
 [2, 2, 0],
 [2, 3, 1],
 [1, 3, 2],
 [4, 3, -1],
 [1, 2, 1],
 [4, 4, 0],
 [0, 2, 2],
 [3, 3, 0],
 [0, 0, 0],
 [4, 0, -4],
 [3, 2, -1],
 [0, 2, 2],
 [3, 3, 0],
 [1, 3, 2],
 [4, 3, -1],
 [4, 3, -1],
 [4, 3, -1],
 [4, 3, -1],
 [2, 2, 0],
 [2, 2, 0],
 [3, 3, 0],
 [3, 3, 0],
 [3, 2, -1],
 [2, 3, 1],
 [0, 3, 3],
 [1, 3, 2],
 [0, 2, 2],
 [2, 3, 1],
 [5, 3, -2],
 [4, 3, -1],
 [5, 3, -2],
 [1, 2, 1],
 [2, 2, 0],
 [2, 3, 1],
 [1, 3, 2],
 [2, 2, 0],
 [0, 2, 2],
 [4, 3, -1],
 [3, 3, 0],
 [3, 3, 0],
 [2, 2, 0],
 [5, 3, -2],
 [1, 2, 1],
 [4, 3, -1],
 [6, 3, -3],
 [3, 3, 0],
 [2, 2, 0],
 [4, 3, -1],
 [4, 3, -1],
 [2, 2, 0],
 [2, 2, 0],
 [2, 2, 0],
 [2, 3, 1],
 [4, 3, -1],
 [4, 3, -1],
 [4, 3, -1],
 [5, 3, -2],
 [2, 3, 1],
 [4, 3, -1],
 [3, 3, 0],
 [2, 3, 1],
 [5, 3, -2],
 [2, 2, 0],
 [0, 2, 2],
 [3, 3, 0],
 [3, 2, -1],
 [1, 2, 1],
 [3, 2, -1],
 [3, 3, 0],
 [7, 3, -4],
 [4, 3, -1],
 [4, 3, -1],
 [2, 2, 0],
 [2, 3, 1],
 [3, 0, -3],
 [3, 2, -1],
 [4, 3, -1],
 [0, 2, 2],
 [7, 3, 

In [33]:
y_pred_logreg_value = [i.index(max(i)) for i in y_pred_logreg]
print('\nScore test: \n', accuracy_score(list(y_test["absenteeism"]), y_pred_logreg_value))
print('\nMean absolute error: \n', metrics.mean_absolute_error(y_test_value, y_pred_logreg_value))
print('\nMedian absolute error: \n', metrics.median_absolute_error(y_test_value, y_pred_logreg_value), "\n")
print(*show_delta(y_test_value, y_pred_logreg_value), sep="\n")


Score test: 
 0.3300589390962672

Mean absolute error: 
 1.1080550098231827

Median absolute error: 
 1.0 

[4, 3, -1]
[1, 2, 1]
[2, 2, 0]
[2, 3, 1]
[1, 3, 2]
[4, 3, -1]
[1, 2, 1]
[4, 4, 0]
[0, 2, 2]
[3, 3, 0]
[0, 0, 0]
[4, 0, -4]
[3, 2, -1]
[0, 2, 2]
[3, 3, 0]
[1, 3, 2]
[4, 3, -1]
[4, 3, -1]
[4, 3, -1]
[4, 3, -1]
[2, 2, 0]
[2, 2, 0]
[3, 3, 0]
[3, 3, 0]
[3, 2, -1]
[2, 3, 1]
[0, 3, 3]
[1, 3, 2]
[0, 2, 2]
[2, 3, 1]
[5, 3, -2]
[4, 3, -1]
[5, 3, -2]
[1, 2, 1]
[2, 2, 0]
[2, 3, 1]
[1, 3, 2]
[2, 2, 0]
[0, 2, 2]
[4, 3, -1]
[3, 3, 0]
[3, 3, 0]
[2, 2, 0]
[5, 3, -2]
[1, 2, 1]
[4, 3, -1]
[6, 3, -3]
[3, 3, 0]
[2, 2, 0]
[4, 3, -1]
[4, 3, -1]
[2, 2, 0]
[2, 2, 0]
[2, 2, 0]
[2, 3, 1]
[4, 3, -1]
[4, 3, -1]
[4, 3, -1]
[5, 3, -2]
[2, 3, 1]
[4, 3, -1]
[3, 3, 0]
[2, 3, 1]
[5, 3, -2]
[2, 2, 0]
[0, 2, 2]
[3, 3, 0]
[3, 2, -1]
[1, 2, 1]
[3, 2, -1]
[3, 3, 0]
[7, 3, -4]
[4, 3, -1]
[4, 3, -1]
[2, 2, 0]
[2, 3, 1]
[3, 0, -3]
[3, 2, -1]
[4, 3, -1]
[0, 2, 2]
[7, 3, -4]
[3, 2, -1]
[4, 2, -2]
[3, 3, 0]
[3, 3, 0]
[3, 2,

---

### Что есть?

1. y_pred_tree - mod - дерево
2. y_pred_lin - model_lin - линейная
3. y_pred_nn - y_pred_nn_value - models_nn - нейронка
4. y_pred_logreg - y_pred_logreg_value - models_logreg - логическая